"""
[V1]
* XGBoost Optuna find parameters
[To-Do]
* LightGBM
* Catboost

ResNeSt:

"""

In [1]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
import numpy as np

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [2]:
df = fetch_california_housing(as_frame=True)["frame"]
print(df.shape)
df.head()

(20640, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [3]:
from sklearn.model_selection import train_test_split

trainX, testX, trainy, testy = train_test_split(df.drop(columns=['MedHouseVal']), df.MedHouseVal, test_size=0.15, shuffle=True, random_state=1337)

In [4]:
%%time
import optuna
import xgboost as xgb
#from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


def obj_xgb(trial, X, y):
    #x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15)
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000, step=50),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 0.001, 0.2),
        'objective': 'reg:squarederror', 
        'eval_metric': 'rmse',
    }

    #
    # custom_scorer = make_scorer(mean_squared_error, greater_is_better=False)
    model = xgb.XGBRegressor(**params)
    #random_state_list = [X, X, X, X, X]
    #cv_score/=len(random_state_list)

    cv = KFold(n_splits=5, shuffle=True, random_state=11)
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        #scores = cross_val_score(model, x, y, cv=kf,scoring=custom_scorer)
        cv_scores[idx] = mean_squared_error(y_test, preds)
 
    avg = cv_scores.mean()
    #y_pred = model.predict_proba(x_valid)[:, 1]
    #roc_auc = roc_auc_score(y_valid, y_pred)
    return avg


study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: obj_xgb(trial, trainX, trainy), n_trials=20)  # Pass X and y to the obj_xgb function

display(study)

print('Best trial:', study.best_trial.params)
print('Best brier:', study.best_trial.value)

[I 2023-08-22 15:40:13,538] A new study created in memory with name: no-name-475bb466-8e96-4749-a0e3-0d5852979116
/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate 

Best trial: {'n_estimators': 950, 'max_depth': 7, 'learning_rate': 0.053205633034397806, 'subsample': 0.9270682514998716, 'colsample_bytree': 0.7921756371948105, 'gamma': 0.014086740287365148}
Best brier: 0.19407107619201097
CPU times: user 56min 32s, sys: 1min 21s, total: 57min 54s
Wall time: 20min 47s


In [9]:
model_xgb = xgb.XGBRegressor()
model_xgb.fit(trainX, trainy)
print('Mean Squared Error(Default):',mean_squared_error(testy, model_xgb.predict(testX)))

optuna_xgb = study.best_trial.params
model_optuna_xgb = xgb.XGBRegressor(**optuna_xgb)
model_optuna_xgb.fit(trainX, trainy)
print('Mean Squared Error(Optuna):',mean_squared_error(testy, model_optuna_xgb.predict(testX)))

/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Mean Squared Error: 0.2124221487617203
Mean Squared Error: 0.1896837939425726


/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [10]:
pd.DataFrame({'predy': model2.predict(testX), 'testy':pd.Series(testy)}).sort_index().head(20)

/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


,predy,testy
2,4.134664,3.521
15,1.338883,1.400
21,1.266553,1.598
30,1.393143,1.223
31,1.261677,1.152
37,1.462393,1.039
45,1.947667,1.823
47,1.402132,1.375
48,1.163201,1.875
52,1.056554,0.975


## 2 - LightGBM

In [5]:
%%time
import optuna
import lightgbm as lgbm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


def obj_lgbm(trial, X, y):
    #x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15)
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    params = {
        "metric": "RMSE",
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01,0.5),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'num_threads': trial.suggest_int('num_threads', 1, 10),
        "verbosity": -1,
        "boosting_type": "gbdt"}

    #
    # custom_scorer = make_scorer(mean_squared_error, greater_is_better=False)
    model = lgbm.LGBMRegressor(**params)
    #random_state_list = [X, X, X, X, X]
    #cv_score/=len(random_state_list)

    cv = KFold(n_splits=5, shuffle=True, random_state=22)
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        #scores = cross_val_score(model, x, y, cv=kf,scoring=custom_scorer)
        cv_scores[idx] = mean_squared_error(y_test, preds)
 
    avg = cv_scores.mean()
    #y_pred = model.predict_proba(x_valid)[:, 1]
    #roc_auc = roc_auc_score(y_valid, y_pred)
    return avg


study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: obj_lgbm(trial, trainX, trainy), n_trials=20)  # Pass X and y to the obj_xgb function

display(study)

print('Best trial:', study.best_trial.params)
print('Best brier:', study.best_trial.value)

[I 2023-08-22 16:10:27,334] A new study created in memory with name: no-name-89718e45-0cce-42fd-bd10-76a2c160f34f


[LightGBM] [Warning] feature_fraction is set=0.5149384832512255, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5149384832512255
[LightGBM] [Warning] num_threads is set=3, n_jobs=-1 will be ignored. Current value: num_threads=3
[LightGBM] [Warning] lambda_l1 is set=0.21551290223960975, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.21551290223960975
[LightGBM] [Warning] bagging_fraction is set=0.34578044275383474, subsample=1.0 will be ignored. Current value: bagging_fraction=0.34578044275383474
[LightGBM] [Warning] lambda_l2 is set=6.979581957195179, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.979581957195179
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] feature_fraction is set=0.5149384832512255, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5149384832512255
[LightGBM] [Warning] num_threads is set=3, n_jobs=-1 will be ignore

[I 2023-08-22 16:10:29,813] Trial 0 finished with value: 0.2300178951730601 and parameters: {'lambda_l1': 0.21551290223960975, 'lambda_l2': 6.979581957195179, 'num_leaves': 141, 'learning_rate': 0.17945351300755408, 'feature_fraction': 0.5149384832512255, 'bagging_fraction': 0.34578044275383474, 'bagging_freq': 14, 'min_child_samples': 43, 'num_threads': 3}. Best is trial 0 with value: 0.2300178951730601.


[LightGBM] [Warning] feature_fraction is set=0.2020026090039036, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2020026090039036
[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be ignored. Current value: num_threads=4
[LightGBM] [Warning] lambda_l1 is set=0.9915571637991767, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9915571637991767
[LightGBM] [Warning] bagging_fraction is set=0.10195191976091343, subsample=1.0 will be ignored. Current value: bagging_fraction=0.10195191976091343
[LightGBM] [Warning] lambda_l2 is set=2.3662621925414354e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.3662621925414354e-05
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.2020026090039036, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2020026090039036
[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be 

[I 2023-08-22 16:10:30,720] Trial 1 finished with value: 0.3829692356125848 and parameters: {'lambda_l1': 0.9915571637991767, 'lambda_l2': 2.3662621925414354e-05, 'num_leaves': 138, 'learning_rate': 0.43088511511006766, 'feature_fraction': 0.2020026090039036, 'bagging_fraction': 0.10195191976091343, 'bagging_freq': 3, 'min_child_samples': 99, 'num_threads': 4}. Best is trial 0 with value: 0.2300178951730601.


...

[LightGBM] [Warning] feature_fraction is set=0.884753780087334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.884753780087334
[LightGBM] [Warning] num_threads is set=10, n_jobs=-1 will be ignored. Current value: num_threads=10
[LightGBM] [Warning] lambda_l1 is set=1.1809220597800195e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1809220597800195e-08
[LightGBM] [Warning] bagging_fraction is set=0.8788975671941567, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8788975671941567
[LightGBM] [Warning] lambda_l2 is set=0.0004268357159036397, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0004268357159036397
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.884753780087334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.884753780087334
[LightGBM] [Warning] num_threads is set=10, n_jobs=-1 will 

[I 2023-08-22 16:12:20,791] Trial 18 finished with value: 0.42647068414783507 and parameters: {'lambda_l1': 1.1809220597800195e-08, 'lambda_l2': 0.0004268357159036397, 'num_leaves': 3, 'learning_rate': 0.055992282538370186, 'feature_fraction': 0.884753780087334, 'bagging_fraction': 0.8788975671941567, 'bagging_freq': 1, 'min_child_samples': 48, 'num_threads': 10}. Best is trial 15 with value: 0.20097476261019226.


[LightGBM] [Warning] feature_fraction is set=0.884753780087334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.884753780087334
[LightGBM] [Warning] num_threads is set=10, n_jobs=-1 will be ignored. Current value: num_threads=10
[LightGBM] [Warning] lambda_l1 is set=1.1809220597800195e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1809220597800195e-08
[LightGBM] [Warning] bagging_fraction is set=0.8788975671941567, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8788975671941567
[LightGBM] [Warning] lambda_l2 is set=0.0004268357159036397, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0004268357159036397
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.9941747944650581, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9941747944650581
[LightGBM] [Warning] num_threads is set=7, n_jobs=-1 will

[I 2023-08-22 16:12:25,126] Trial 19 finished with value: 0.47387547052683165 and parameters: {'lambda_l1': 1.1561591986290942e-05, 'lambda_l2': 0.015947743454758405, 'num_leaves': 50, 'learning_rate': 0.01065601635146983, 'feature_fraction': 0.9941747944650581, 'bagging_fraction': 0.9973105947151126, 'bagging_freq': 0, 'min_child_samples': 19, 'num_threads': 7}. Best is trial 15 with value: 0.20097476261019226.


Best trial: {'lambda_l1': 4.12348857087004e-06, 'lambda_l2': 0.005500580751154387, 'num_leaves': 100, 'learning_rate': 0.09706783767482643, 'feature_fraction': 0.7069249789707748, 'bagging_fraction': 0.8329693941192573, 'bagging_freq': 6, 'min_child_samples': 25, 'num_threads': 7}
Best brier: 0.20097476261019226
CPU times: user 2min 5s, sys: 2min 27s, total: 4min 32s
Wall time: 1min 59s


In [10]:
model_lgbm = lgbm.LGBMRegressor()
model_lgbm.fit(trainX, trainy)
print('Mean Squared Error(Default):',mean_squared_error(testy, model_lgbm.predict(testX)))

#optuna_xgb = study.best_trial.params
optuna_lgbm = {'lambda_l1': 4.12348857087004e-06, 'lambda_l2': 0.005500580751154387, 'num_leaves': 100, 'learning_rate': 0.09706783767482643, 'feature_fraction': 0.7069249789707748, 'bagging_fraction': 0.8329693941192573, 'bagging_freq': 6, 'min_child_samples': 25, 'num_threads': 7}
model_optuna_lgbm = lgbm.LGBMRegressor(**optuna_lgbm)
model_optuna_lgbm.fit(trainX, trainy)
print('Mean Squared Error(Optuna):',mean_squared_error(testy, model_optuna_lgbm.predict(testX)))

Mean Squared Error(Default): 0.20981720847571264
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] num_threads is set=7, n_jobs=-1 will be ignored. Current value: num_threads=7
[LightGBM] [Warning] lambda_l1 is set=4.12348857087004e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.12348857087004e-06
[LightGBM] [Warning] feature_fraction is set=0.7069249789707748, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7069249789707748
[LightGBM] [Warning] bagging_fraction is set=0.8329693941192573, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8329693941192573
[LightGBM] [Warning] lambda_l2 is set=0.005500580751154387, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.005500580751154387
Mean Squared Error(Optuna): 0.18964731561500517


# 3 - cat

In [8]:
%%time
import optuna
import catboost as cat
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


def obj_cat(trial, X, y):
    #x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15)
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    params = {'iterations':trial.suggest_int("iterations", 1000, 12000),
              'od_wait':trial.suggest_int('od_wait', 500, 2000),
              'loss_function':'RMSE',
              #'task_type':"GPU",
              'eval_metric':'RMSE',
              'learning_rate' : trial.suggest_uniform('learning_rate',0.01,0.3),
              'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
              'subsample': trial.suggest_uniform('subsample',0,1),
              'random_strength': trial.suggest_uniform('random_strength',1,50),
              'depth': trial.suggest_int('depth',3,14),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
              'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
               }

    #
    # custom_scorer = make_scorer(mean_squared_error, greater_is_better=False)
    model = cat.CatBoostRegressor(**params)
    #random_state_list = [X, X, X, X, X]
    #cv_score/=len(random_state_list)

    cv = KFold(n_splits=5, shuffle=True, random_state=22)
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train, y_train, eval_set=[(X_test,y_test)], early_stopping_rounds=100, verbose=False)
        preds = model.predict(X_test)
        #scores = cross_val_score(model, x, y, cv=kf,scoring=custom_scorer)
        cv_scores[idx] = mean_squared_error(y_test, preds)
 
    avg = cv_scores.mean()
    #y_pred = model.predict_proba(x_valid)[:, 1]
    #roc_auc = roc_auc_score(y_valid, y_pred)
    return avg


study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: obj_cat(trial, trainX, trainy), n_trials=20)  # Pass X and y to the obj_xgb function

display(study)

print('Best trial:', study.best_trial.params)
print('Best brier:', study.best_trial.value)

[I 2023-08-22 16:18:40,827] A new study created in memory with name: no-name-89ace89a-0540-42c4-ae31-8c8aae86d75b


Custom logger is already specified. Specify more than one logger at same time is not thread safe.[I 2023-08-22 16:23:56,538] Trial 0 finished with value: 0.2240154986089194 and parameters: {'iterations': 10009, 'od_wait': 1382, 'learning_rate': 0.013740478200540295, 'reg_lambda': 10.123236300802672, 'subsample': 0.14081343425029347, 'random_strength': 7.506661848714563, 'depth': 3, 'min_data_in_leaf': 29, 'leaf_estimation_iterations': 11}. Best is trial 0 with value: 0.2240154986089194.
[I 2023-08-22 17:20:10,388] Trial 1 finished with value: 0.2373425808496406 and parameters: {'iterations': 7879, 'od_wait': 1461, 'learning_rate': 0.12472417251474666, 'reg_lambda': 0.1473880138552897, 'subsample': 0.46481213627906026, 'random_strength': 39.08152457442036, 'depth': 13, 'min_data_in_leaf': 15, 'leaf_estimation_iterations': 8}. Best is trial 0 with value: 0.2240154986089194.
[I 2023-08-22 17:52:55,904] Trial 2 finished with value: 0.22177510194134467 and parameters: {'iterations': 3666, '

Best trial: {'iterations': 11907, 'od_wait': 1936, 'learning_rate': 0.1899099511160147, 'reg_lambda': 67.66690655367276, 'subsample': 0.9376510157656168, 'random_strength': 20.855157436589423, 'depth': 7, 'min_data_in_leaf': 11, 'leaf_estimation_iterations': 2}
Best brier: 0.19443931612727425
CPU times: user 2h 45min 21s, sys: 1h 33min 30s, total: 4h 18min 52s
Wall time: 2h 31min 14s


In [12]:
model_cat = cat.CatBoostRegressor()
model_cat.fit(trainX, trainy)
print('Mean Squared Error(Default):',mean_squared_error(testy, model_cat.predict(testX)))

optuna_cat = study.best_trial.params
#optuna_cat = Best trial: {'iterations': 11907, 'od_wait': 1936, 'learning_rate': 0.1899099511160147, 'reg_lambda': 67.66690655367276, 'subsample': 0.9376510157656168, 'random_strength': 20.855157436589423, 'depth': 7, 'min_data_in_leaf': 11, 'leaf_estimation_iterations': 2}
model_optuna_cat = cat.CatBoostRegressor(**optuna_cat)
model_optuna_cat.fit(trainX, trainy)
print('Mean Squared Error(Optuna):',mean_squared_error(testy, model_optuna_cat.predict(testX)))

Learning rate set to 0.064991
0:	learn: 1.1140822	total: 15.9ms	remaining: 15.9s
1:	learn: 1.0772709	total: 23.9ms	remaining: 11.9s
2:	learn: 1.0423054	total: 31.4ms	remaining: 10.4s
3:	learn: 1.0099463	total: 43.5ms	remaining: 10.8s
4:	learn: 0.9814220	total: 54.8ms	remaining: 10.9s
5:	learn: 0.9541797	total: 66.4ms	remaining: 11s
6:	learn: 0.9276795	total: 79.8ms	remaining: 11.3s
7:	learn: 0.9036231	total: 94ms	remaining: 11.7s
8:	learn: 0.8823419	total: 110ms	remaining: 12.1s
9:	learn: 0.8600750	total: 124ms	remaining: 12.2s


10:	learn: 0.8412545	total: 207ms	remaining: 18.6s
11:	learn: 0.8232505	total: 215ms	remaining: 17.7s
12:	learn: 0.8076310	total: 232ms	remaining: 17.6s
13:	learn: 0.7925014	total: 238ms	remaining: 16.7s
14:	learn: 0.7784209	total: 245ms	remaining: 16.1s
15:	learn: 0.7663049	total: 261ms	remaining: 16s
16:	learn: 0.7531270	total: 271ms	remaining: 15.7s
17:	learn: 0.7418711	total: 285ms	remaining: 15.5s
18:	learn: 0.7328313	total: 294ms	remaining: 15.2s
19:	learn: 0.7235767	total: 304ms	remaining: 14.9s
20:	learn: 0.7153682	total: 316ms	remaining: 14.7s
21:	learn: 0.7070522	total: 326ms	remaining: 14.5s
22:	learn: 0.6982236	total: 332ms	remaining: 14.1s
23:	learn: 0.6887391	total: 342ms	remaining: 13.9s
24:	learn: 0.6824850	total: 351ms	remaining: 13.7s
25:	learn: 0.6761531	total: 510ms	remaining: 19.1s
26:	learn: 0.6680861	total: 518ms	remaining: 18.7s
27:	learn: 0.6608141	total: 527ms	remaining: 18.3s
28:	learn: 0.6543494	total: 534ms	remaining: 17.9s
29:	learn: 0.6474590	total: 658ms

In [13]:
print('Mean Squared Error(Default):',mean_squared_error(testy, model_cat.predict(testX)))
print('Mean Squared Error(Optuna):',mean_squared_error(testy, model_optuna_cat.predict(testX)))

Mean Squared Error(Default): 0.18858924229301247
Mean Squared Error(Optuna): 0.19704126878404296
